In [1]:
import ROOT
from ROOT import TF1, TCanvas, TMath


%jsroot on

ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-linux-gnu-g++-13   -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0


在$_{}^{137}Cs$放射源旁放置$LaBr_{3}(Ce)$探测器，利用定标器记录每秒间隔内探测器的计数。通过改变探测器与放射源的距离，得到两组不同计数率下的实验数据。其中一组探测器离放射源较近(high_counts.txt)，另一组探测器离放射源较远(low_counts.txt)。

In [2]:
high_tf1f = ROOT.TH1F("high", "hist", 200, 400,900)  
low_tf1f = ROOT.TH1F("low", "hist", 15, 0, 15)

In [3]:
#打开两个文件，分别读取数据并填充到两个直方图中
with open("high_counts.txt", 'r') as file:
    for line in file:
        parts = line.split()  # 按空格分割每行
        if len(parts) == 2: 
            high_tf1f.Fill(float(parts[1]))  # 假设每行有两个部分，第二个部分是我们要的值
with open("low_counts.txt", 'r') as file:
    for line in file:
        parts = line.split()
        if len(parts) == 2:
            low_tf1f.Fill(float(parts[1]))

In [4]:
c1 = ROOT.TCanvas()            #绘图
c1.Divide(2,1)
c1.cd(1)
high_tf1f.Draw()
c1.cd(2)
low_tf1f.Draw()
c1.Draw()
c1.cd(1)

## 使用高斯拟合数据

In [5]:
c1.cd(1);
high_tf1f.Fit("gaus");
c1.cd(2);
low_tf1f.Fit("gaus");
c1.Draw();

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      383.578
NDf                       =           73
Edm                       =  3.19772e-06
NCalls                    =           63
Constant                  =      303.431   +/-   4.24276     
Mean                      =      665.834   +/-   0.292029    
Sigma                     =      25.6615   +/-   0.211547     	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      131.666
NDf                       =            8
Edm                       =  1.85047e-07
NCalls                    =           59
Constant                  =      1815.21   +/-   27.1154     
Mean                      =       3.3562   +/-   0.0220547   
Sigma                     =      1.82929   +/-   0.0211487    	 (limited)


根据高斯拟合数据的结果，较近的探测器中，$\mu =665.988,\sigma = 25.676，\sqrt{\mu} =25.806 $，
较近的探测器中，$\mu =3.35620,\sigma = 1.82929，\sqrt{\mu} =1.83199$

## 使用泊松分布拟合数据

泊松分布为$$P(x)=a_{0}\frac{\lambda^x e^{-\lambda}}{x!}$$

In [6]:


#构建拟合函数
possi_high = TF1("possi_high", "[0]*TMath::Power(x, [1])*(TMath::Exp(-[1])/TMath::Gamma(x))", 0, 1000)
#possi_high_1 = TF1("possi_high_1", "1200*TMath::Power(x, 665)*(TMath::Exp(-665)/TMath::Gamma(x))", 1, 1000)
possi_low = TF1("possi_low", "[0]*TMath::Power(x, [1])*(TMath::Exp(-[1])/TMath::Gamma(x))", 0, 20)
possi = TF1("possi", TMath.Poisson)

# 设置参数名
possi_high.SetParNames("Constant", "Mean")
possi_low.SetParNames("Constant", "Mean")

#新建一个画布
c2 = TCanvas("c2", "Canvas with Two Pads", 800, 600)
c2.Clear()
c2.Divide(2, 1)


c2.cd(1)
high_tf1f.Draw()  
high_tf1f.Fit("TMath.PoissonI", "R");  


c2.cd(2)
low_tf1f.Fit("TMath.PoissonI", "R");  
low_tf1f.Draw()

c2.Update()


Unknown function: TMath.PoissonI
Unknown function: TMath.PoissonI


In [7]:
TMath.Gaus

In [8]:

possi_high_1 = TF1("possi_high_1", "TMath::PoissonI([0], [1])",400,10000,2)
possi_high_1.SetParNames("Mean", "Constant")

c2.cd(1)
high_tf1f.Draw()  
high_tf1f.Fit(possi_high_1);  
c2.Draw()

****************************************
         Invalid FitResult  (status = 3 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      8047.85
NDf                       =           74
NCalls                    =           60
Mean                      =            0   +/-   -nan        
Constant                  =            0   +/-   -nan        


Warning in <Fit>: Abnormal termination of minimization.


从拟合结果来看，对于高计数率的数据，无法使用ROOT中的TF1函数进行拟合；对于低计数率的数据，使用TF1函数进行拟合可以得到较好的结果。
因此我们尝试提取相关的数据并用python相关库进行拟合。

2.在$_{}^{137}Cs$放射源旁放置$LaBr_{3}(Ce)$探测器，记录每秒间隔内探测器的计数, 并记录相邻脉冲的时间间隔(时间单位$\mu s$) 。

In [15]:
from ROOT import TH1F, TCanvas
import numpy as np

# Create histogram
counts_tf1f = TH1F("counts", "Histogram of Counts per Second", 100, 600., 1000.)

# Read data from file and fill histogram
with open("counts_per_second.txt") as counts_per_second:
    for line in counts_per_second:
        R = float(line.strip())  # Convert each line to a float
        counts_tf1f.Fill(R)

# Create canvas and draw histogram
c3 = TCanvas("c3", "Counts Histogram", 800, 600)
counts_tf1f.Draw()
c3.Update()  # Update the canvas to display


In [16]:
from ROOT import TH1F, TCanvas
import numpy as np

# 创建两个直方图
times_tf1f = TH1F("times", "Histogram of Times", 100, 0., 10000.)
times5_tf1f = TH1F("times5", "Histogram of Times (Grouped by 4)", 100, 0., 50000.)

# 读取文件并填充直方图
count5 = 0
n = 0 

with open("times_between_sca_pulses.txt") as times_between_sca_pulses:
    for line in times_between_sca_pulses:
        R = float(line.strip())  # 每行数据转换为浮点数
        n += 1
        times_tf1f.Fill(R)  # 填充到第一个直方图
        count5 += R  # 累积到 count5
        
        if n == 4:  # 每四个数据填充到第二个直方图
            times5_tf1f.Fill(count5)
            n = 0
            count5 = 0.

# 创建画布并绘制两个直方图
c4 = TCanvas("c4", "Canvas with Two Pads", 800, 600)
c4.Divide(2, 1)

# 绘制第一个直方图
c4.cd(1)
times_tf1f.Draw()

# 绘制第二个直方图
c4.cd(2)
times5_tf1f.Draw()

# 更新画布显示
c4.Update()


In [17]:
c4.cd(1)
times_tf1f.Fit("expo");  #fit with function of f(x) = exp(p0+p1*x)
c4.cd(2)
times5_tf1f.Fit("expo");
c4.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      141.593
NDf                       =           97
Edm                       =   2.6206e-11
NCalls                    =           53
Constant                  =      8.61688   +/-   0.00535232  
Slope                     = -0.000813157   +/-   3.03968e-06 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      15697.6
NDf                       =           37
Edm                       =  5.71739e-08
NCalls                    =           67
Constant                  =       4.5568   +/-   0.0321183   
Slope                     = -0.000134274   +/-   2.76616e-06 
